In [39]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#url = 'https://onepetro.org/search-results?fl_SiteID=1&qb=%7b%22Title1%22%3a%22hydrogen%22%7d&page=1'
#'''
url_list = []
for i in range(1, 41):
    url = f'https://onepetro.org/search-results?fl_SiteID=1&qb=%7b%22Title1%22%3a%22hydrogen%22%7d&page={i}'
    url_list.append(url)
url_list
#'''

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
data = []
for url in url_list:
    page = requests.get(url, headers=headers)
    data.append(page.text)

#soup = BeautifulSoup(page.content, 'html.parser')
#print(soup.prettify())

s = ''.join(data)
with open('data\\raw_data.html', mode='w', encoding='UTF-8', errors='strict', buffering=1) as f:
    f.write(s)

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

with open("data\\raw_data.html", encoding='UTF-8', errors='strict') as f:
    soup = BeautifulSoup(f)

In [101]:
paper_list = []
#paper = soup.find('div', {'class': 'item-container'})
for paper in soup.find_all('div', {'class': 'item-container'}):
    title = paper.find('h4').text.strip()
    
    authors = paper.find('div', {'class': 'sri-authors al-authors-list'}).text
    authors = authors.split(',')
    authors = [x.strip() for x in authors]

    doi = paper.find('div', {'class': 'ww-citation-wrap-doi'}).text
    doi = doi.split(':')[1].strip()

    try:
        publisher = paper.find('div', {'class': 'sri-publisher-name'}).text
    except:
        publisher = paper.find('div', {'class': None}).text
    publisher = publisher.split(':')[1].strip()

    try:
        date = paper.find('div', {'class': 'al-citation-list'}).text.split(',')
        date = (date[-2] + date[-1]).strip()
        chars = list(date.split(' ')[1])
        if len(chars) == 3:
            day = chars[0]
        if len(chars) == 5:
            day = chars[0] + chars[1]
        year = date.split(' ')[-1]
        month = date.split(' ')[0]
        date = ' '.join([day, month, year])
    except:
        date = paper.find('div', {'class': 'sri-date al-pub-date'}).text.split(':')
        date = date[1].strip()

    abstract = paper.find('div', {'class': 'abstract-response-placeholder hide'}).text
    
    paper_list.append((title, authors, doi, publisher, date, abstract))
 

In [153]:
df = pd.DataFrame(paper_list)
df.columns = ['title', 'authors', 'doi', 'publisher', 'date','abstract']
df['date'] = pd.to_datetime(df['date'])
def test(authors):
    short_names = []
    for author in authors:
        author = author[0] + '. ' + author.split(' ')[-1]
        short_names.append(author)
    return short_names

df['authors'] = df['authors'].apply(test)
df

,title,authors,doi,publisher,date,abstract
0,A Blowout Accident Causative Model of Hydrogen...,"[C. Han, Z. Guan, Y. Xu, H. Hu, D. Wu]",SPE-201032-MS,Society of Petroleum Engineers (SPE),2021-06-08,Blowout is one of the most serious accidents i...
1,Transporting Hydrogen-Natural Gas Mixtures,"[R. Kurz, M. Lubomirsky, R. Zamotorin]",PSIG-2110,Pipeline Simulation Interest Group,2021-05-03,"Abstract Both in Europe and in North America, ..."
2,A Fracture Mechanics Approach to Characterizin...,"[H. Amaya, B. Fahimi, R. Thodla]",NACE-2021-16798,NACE International,2021-04-19,Abstract This paper elaborates on a method pre...
3,Insights into Hydrogen Embrittlement (HE) Susc...,"[T. Das, J. Song, S. Brahimi, S. Yue]",NACE-2021-16887,NACE International,2021-04-19,Abstract Microstructure and strength affect hy...
4,The Influence of Stress Concentration and Plas...,"[D. Griffiths, K. Sotoudeh, M. Dodge, M. Gittos]",NACE-2021-16644,NACE International,2021-04-19,Abstract Precipitation-hardened nickel alloys ...
...,...,...,...,...,...,...
785,Results of Hydrogenation Research Work in the ...,[H. Waterman],WPC-154,World Petroleum Congress,1933-07-18,SUMMARY HYDROGENATION processes are often comb...
786,Properties of Hydrogenated Motor Oils.,"[R. Haslam, R. Russell, W. Asbury]",WPC-151,World Petroleum Congress,1933-07-18,INTRODUCTION Tim history of the development of...
787,Comparison of Cracking and Hydrogenation as Me...,"[R. Haslam, R. Russell, W. Asbury]",WPC-152,World Petroleum Congress,1933-07-18,INTRODUCTION THE general applicability of the ...
788,An Historical Account of Hydrogenation.,[F. Bergius],WPC-148,World Petroleum Congress,1933-07-18,"TaF, research and technical work on hydrogenat..."


In [154]:
%matplotlib qt
import matplotlib.pyplot as plt
plt.ion()
fig = plt.figure(figsize=(14, 8))
ax = fig.add_subplot(111)

plt.title('Pupblikationen über Wasserstoff pro Jahr')
plt.plot(df['date'].dt.year.value_counts().sort_index())

In [155]:
df['publisher'].value_counts()

NACE International                                               453
Society of Petroleum Engineers (SPE)                             152
World Petroleum Congress                                          76
International Society of Offshore and Polar Engineers             51
Offshore Technology Conference                                    22
Petroleum Society of Canada                                       10
International Petroleum Technology Conference                      8
Oil Industry Journal                                               5
American Society of Safety Professionals                           4
Society of Petrophysicists and Well-Log Analysts                   2
International Society for Rock Mechanics and Rock Engineering      2
Society of Exploration Geophysicists                               1
Pipeline Simulation Interest Group                                 1
Society of Naval Architects and Marine Engineers                   1
Society of Underwater Technology  

In [156]:
from  itertools import chain
from collections import Counter

s = pd.Series(list(chain.from_iterable(df['authors'])))
s.str.strip().value_counts()

F. Dean          15
K. Kim           13
L. Qiao          11
J. Jr.           11
R. Johnsen        9
                 ..
B. Kumar          1
J. Johnson        1
M. Lubomirsky     1
J. Shick          1
Z. Yonglan        1
Length: 1814, dtype: int64

In [169]:
pd.set_option('display.max_rows', 500)

df[df['authors'].apply(lambda x: 'F. Dean' in x)]['title'].to_list()

['Multipoint Flux Monitoring by the Hydrogen Collection Method Using a New Wide Temperature Range Flux Measurement Probe',
 'Hydrogen Cracking Behavior of Legacy Carbon Steels',
 'Detection And Monitoring Of Naphthenic Acid Corrosion In A Visbreaker Unit Using Hydrogen Flux Measurements',
 'A Review Of Hydrogen Flux Promoters',
 'Correlation Of Hydrogen Flux And Corrosion Rate Measurements Carried Out During A Severe Episode Of Corrosion-Erosion Attributable',
 'Hydrogen Flux And Corrosion Rate Measurements On Hydrogen Induced Cracking Susceptible And Resistant A516 Steels In Various Sour Environments',
 'The Effect Of Pressure On Hydrogen Charging Of Carbon Steel Under Cathodic Protection',
 'Hydrogen Flux and High Temperature Acid Corrosion',
 'Practical Applications of Hydrogen Permeation Monitoring',
 'Examples of Use and Interpretation of Field Data Using a Portable Hydrogen Permeation Monitor',
 'Applications of Hydrogen Flux Monitoring to Pre-weld Bakeouts of Steel',
 'A Laborat

In [164]:
df_storage = df[df['abstract'].str.contains('cavern')]
df_storage['title'].to_list()

['Development of a Supporting Tool for the Evaluation of Storage Capacity of Caverns for Renewable Energies – Rock Mechanical Design for Hydrogen Storage Caverns',
 'The H2STORE Project - Experimental and Numerical Simulation Approach to Investigate Processes in Underground Hydrogen Reservoir Storage']